In [2]:
using Random
using SparseArrays
using LinearAlgebra
using BenchmarkTools

# Sigmoid function
sigmoid(x) = 1 / (1 + exp(-x))

# Set random seed
rng = MersenneTwister(1)

n, d, k = 1_000_000, 10, 800
X = randn(rng, n, d)
beta = randn(rng, d)
g = rand(rng, 1:k, n)

# Construct G matrix and Y from the groups g and the data X
data = ones(n)
indices = hcat(1:n, g)
G = sparse(indices[:,1], indices[:,2], data, n, k)'
Y = G * sigmoid.(X * beta)

800-element Vector{Float64}:
 662.6942976987755
 618.4764344469371
 612.1582081961544
 614.2572157292472
 655.1992252045744
 615.1134814771395
 608.1138685515541
 599.6363849927284
 670.2068321240985
 608.8750027638932
   ⋮
 625.0953992137233
 649.6584266431195
 617.2160336720075
 594.6136767234705
 591.3045195746392
 637.7551374911894
 621.0331497398638
 647.5278389830117
 616.18172902362

In [41]:
function sparse_loss_fn(beta)
    return mean((G * sigmoid.(X * beta) .- Y).^2, dims=1)
end

sparse_loss_fn (generic function with 1 method)

In [42]:
sparse_loss_fn(params .* 0)

1×100 Matrix{Float64}:
 237.665  237.665  237.665  237.665  …  237.665  237.665  237.665  237.665

In [4]:
params = randn(rng, d)
grad = gradient(sparse_loss_fn, beta*0.)
# hess = hessian(sparse_loss_fn, beta*0.)

UndefVarError: UndefVarError: `gradient` not defined

In [1]:
using ForwardDiff, ReverseDiff

In [50]:
# Gradient function using ForwardDiff
grad_fn(beta) = ForwardDiff.jacobian(b -> sparse_loss_fn(b), beta)
# grad_fn(beta) = ReverseDiff.gradient(b -> sparse_loss_fn(b), beta)

# Hessian function using Jacobian of the gradient function
hessian_fn(beta) = ForwardDiff.jacobian(grad_fn, beta)

hessian_fn (generic function with 1 method)

In [51]:
params = randn(rng, (d, 100))
hessian_fn(params)

In [ ]:
params = randn(rng, (d, 100))
hessian_fn(params)

In [12]:
params = randn(rng, d)
@benchmark hessian_fn($params)

BenchmarkTools.Trial: 25 samples with 1 evaluation.
 Range (min … max):  196.303 ms … 219.948 ms  ┊ GC (min … max): 1.43% … 6.13%
 Time  (median):     206.446 ms               ┊ GC (median):    6.50%
 Time  (mean ± σ):   207.670 ms ±   5.079 ms  ┊ GC (mean ± σ):  6.26% ± 1.05%

                       ▃█ ▃ ▃█          ▃                     ▃  
  ▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▇▇██▇█▇██▁▁▇▁▁▇▇▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▇▁▁▁▁▁▁▁▁█ ▁
  196 ms           Histogram: frequency by time          220 ms <

 Memory estimate: 1.59 GiB, allocs estimate: 80.

In [43]:
params = randn(rng, (100, d))

100×10 Matrix{Float64}:
 -0.602326  -0.0310429   0.682521     …  -1.19938     0.784479  -0.0979281
  0.439605   1.98135    -0.564195         2.14695    -0.144549   1.03334
  0.214994  -0.370731    0.000905333     -1.19772    -0.553867   0.285461
 -0.341322   1.05155     1.71632          1.36741     0.144165   0.937683
  1.03744    0.431556   -0.107349        -1.60741     1.1509    -0.463932
  0.633397  -0.0883078  -0.594694     …  -0.497014    0.180204   0.571343
 -0.261893  -0.460954    1.05734         -0.525668    0.283683  -1.75742
 -1.21796   -0.455816    0.452604        -0.0846535  -0.402552  -0.0730914
  1.65319    0.328531    0.346625        -0.323957    0.231393  -1.5412
  0.710281  -0.589103   -1.14473          0.620849   -0.468667   1.38703
  ⋮                                   ⋱                         
  0.119559  -0.957195   -0.286772        -0.540889   -0.278937   0.0283729
  0.26452   -0.999995    1.88445          0.573313    0.42162    1.40595
 -2.08181   -0.238089    0

InterruptException: InterruptException:

In [18]:
combinedims(map(hessian_fn, splitdims(params)))

10×10×100 Array{Float64, 3}:
[:, :, 1] =
 36.7245      4.00898    3.70059  …    5.72025  -1.14681    1.6965
  4.00898    10.2771    -4.17017     -23.2168   -5.16176   -4.48953
  3.70059    -4.17017   32.2055       -4.81301  -9.28884    2.56492
 -4.63648     0.721872  -1.24823      -1.29031   0.455441   2.0115
  2.88783   -13.4894    -8.99396      -7.78452  -3.45405    2.83877
 -0.545859    4.0106    -2.03703  …   11.9496    4.8398     2.9264
  0.210985   -3.52073   -1.22296      -1.59566   0.945897   4.54254
  5.72025   -23.2168    -4.81301      17.9613   -5.12465   -2.47633
 -1.14681    -5.16176   -9.28884      -5.12465  38.0845     2.71252
  1.6965     -4.48953    2.56492      -2.47633   2.71252   30.3502

[:, :, 2] =
  8.08067   -6.30789   9.39886   …    3.20356     0.918254  -1.94107
 -6.30789   10.3493    2.87554       -3.09473     3.02206    1.61251
  9.39886    2.87554   5.23886       -0.256905   -5.36278   -5.89122
  6.90496    2.78602  -7.28283       -4.38843    -4.23962   -4.

In [28]:
hessian_fn.(eachrow(params))

100-element Vector{Matrix{Float64}}:
 [16.523745630824568 -4.6953282538450765 … 1.5638174070204032 -2.095916997853869; -4.695328253845178 8.657300043715523 … 12.590106455442822 1.0923772048364302; … ; 1.5638174070204314 12.590106455442005 … -1.1493041090198277 -5.804361016964054; -2.0959169978540433 1.0923772048364013 … -5.804361016964088 11.82033818306524]
 [13.056847654459544 -2.3024338254102 … -1.9466051069024148 9.376021838983707; -2.302433825410207 16.66279211425452 … -0.15461548723436305 3.529231998054975; … ; -1.9466051069023271 -0.1546154872343323 … 28.913395497825324 6.836305379958778; 9.376021838983924 3.5292319980551077 … 6.836305379959158 -5.034997877820383]
 [7.757946949836845 -1.1679881753242054 … -9.572152334085358 -6.554612563630759; -1.1679881753242491 6.238703519512129 … -6.115279873801038 0.71984400994688; … ; -9.572152334085386 -6.115279873801363 … -1.7520807009998884 2.598284358639915; -6.554612563631305 0.7198440099469344 … 2.5982843586399036 5.296856544609978]
 [

In [29]:
@benchmark hessian_fn.(eachrow($params))

BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 20.992 s (7.18% GC) to evaluate,
 with a memory estimate of 158.75 GiB, over 8308 allocations.

In [21]:
# Gradient and Hessian functions
sparse_grad = gradient(sparse_loss_fn, beta, X, Y, G)
sparse_hess = hessian(sparse_loss_fn, beta, X, Y, G)

MethodError: MethodError: no method matching sparse_loss_fn(::Vector{Float64}, ::Matrix{Float64}, ::Vector{Float64}, ::Adjoint{Float64, SparseMatrixCSC{Float64, Int64}})

Closest candidates are:
  sparse_loss_fn(::Any)
   @ Main ~/Dropbox/elrpy/Untitled-1.ipynb:1
